In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import train_test_split
from scipy.optimize import curve_fit
import time

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
torch.manual_seed(12)

import sys
# sys.path.append('../modules')
from mc_ising2d import IsingMC
from net import Net

T_c = 2 / ( np.log(1 + np.sqrt(2)) )

### load Ising data

In [2]:
ising_data = np.load('rbm_data_25_temps.npy')

### Need to label configurations with their temps for supervised learning 

In [3]:
np.shape(ising_data)

(25, 1000, 100)

### flatten the data set so it has 25000 rows of configurations

In [4]:
ising_data = ising_data.reshape((25000, 100))
ising_data


array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [0, 0, 1, ..., 1, 1, 1],
       [1, 0, 0, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 0, 0]])

### label the temperature configurations

In [5]:
df = pd.DataFrame(ising_data)

# Convert rows to list and put them into a single column
df['vector'] = df.apply(lambda row: row.tolist(), axis=1)

# Keep only the vector column
df = df[['vector']]
df

,vector
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...
24995,"[0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, ..."
24996,"[1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, ..."
24997,"[0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, ..."
24998,"[1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, ..."


In [6]:
# Define the temperatures
temperatures = [0.000001, 0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 2.25, 2.5, 2.75, 
                3.0, 3.25, 3.5, 3.75, 4.0, 4.25, 4.5, 4.75, 5.0, 5.25, 5.5, 5.75, 6.0]

# Repeat each temperature 1000 times
temp_column = np.repeat(temperatures, 1000)

# Add the temperature column to the DataFrame
df['temperature'] = temp_column
df = df.rename(columns={'vector': 'spin configuration'})

df

,spin configuration,temperature
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.000001
1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.000001
2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.000001
3,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.000001
4,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.000001
...,...,...
24995,"[0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, ...",6.000000
24996,"[1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, ...",6.000000
24997,"[0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, ...",6.000000
24998,"[1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, ...",6.000000


In [7]:
IsingMC(L= 10).plot_thermodynamics(spin_MC= ising_data, Tc_scaled= False)

Method fit will construct states to the following temperatures: 
[3.5        3.4        3.3        3.2        3.1        3.
 2.9        2.8        2.7        2.6        2.5        2.4
 2.3        2.27918531 2.26918531 2.25918531 2.2        2.1
 2.         1.9        1.8        1.7        1.6        1.5
 1.4        1.3        1.2        1.1        1.        ]


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices